# initial config

In [12]:
host = "10.41.105.190"
port = 830
user = "admin_user"
password = "Ericsson1234"
ROW_LENGTH = 160

import traceback
import xml.dom.minidom
from ncclient import manager
from ncclient.xml_ import to_ele

def print_formatted(string):
    print("printing formatted str: " + str(string))
    try:
        print(xml.dom.minidom.parseString(string).toprettyxml())
    except Exception:
        print("Exception while printing xml dom")
        traceback.print_exc()
        print("Trying to print simplified version")
        print_simplified(string)


def print_simplified(string):
    row = ""
    index = 0
    if len(string) < ROW_LENGTH:
        print(string)
    else:
        for ch in string:
            if index < ROW_LENGTH:
                row += (ch)
                index += 1
            else:
                print(row)
                index = 0
                row = ""

m = manager.connect(host=host, port=port, username=user, password=password, hostkey_verify=False)

In [25]:
message = '''<nc:get xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0">
  <nc:filter type="subtree">
    <if:interfaces xmlns:if="urn:ietf:params:xml:ns:yang:ietf-interfaces"/>
  </nc:filter>
</nc:get>'''
response = m.dispatch(to_ele(message))
print_formatted(response.xml)

TimeoutExpiredError: ncclient timed out while waiting for an rpc reply.

### Here first try to recall the difference between the get and get-config operations. Q: Which datastore(s) are accessed in these operations?
from netconf rfc:

```txt
The <get-config> operation retrieves
   configuration data only, while the <get> operation retrieves
   configuration and state data.
```

```txt
     +-------------+                 +-----------+
     | <candidate> |                 | <startup> |
     |  (ct, rw)   |<---+       +--->| (ct, rw)  |
     +-------------+    |       |    +-----------+
            |           |       |           |
            |         +-----------+         |
            +-------->| <running> |<--------+
                      | (ct, rw)  |
                      +-----------+
                            |
                            |        // configuration transformations,
                            |        // e.g., removal of nodes marked as
                            |        // "inactive", expansion of
                            |        // templates
                            v
                      +------------+
                      | <intended> | // subject to validation
                      | (ct, ro)   |
                      +------------+
                            |        // changes applied, subject to
                            |        // local factors, e.g., missing
                            |        // resources, delays
                            |
       dynamic              |   +-------- learned configuration
       configuration        |   +-------- system configuration
       datastores -----+    |   +-------- default configuration
                       |    |   |
                       v    v   v
                    +---------------+
                    | <operational> | <-- system state
                    | (ct + cf, ro) |
                    +---------------+

     ct = config true; cf = config false
     rw = read-write; ro = read-only
     boxes denote named datastores
```

### Q: Identify at least one field in the response which is config-false (see the Yang model for help). Would this field appear in the response, if we performed a get-config operation instead of the get?

```yang
          container bridge-vlan {
            when "not(derived-from-or-self(../../bridge-type, 'two-port-mac-relay-bridge'))" {
              description
                "Applies to non TPMRs.";
            }
            reference
              "12.10 of IEEE Std 802.1Q-2022";

            leaf version {
              type uint16;
              config false;
              description
                "The version number supported.";
              reference
                "12.10.1.3 of IEEE Std 802.1Q-2022";

            }
```
```txt
7.1.  <get-config>

   Description:  Retrieve all or part of a specified configuration
      datastore.

   Parameters:
```
```txt
7.7.  <get>

   Description:  Retrieve running configuration and device state
      information.

   Parameters:
```



In [15]:
message = '''<nc:get xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0" xmlns:if="urn:ietf:params:xml:ns:yang:ietf-interfaces" xmlns:ianaift="urn:ietf:params:xml:ns:yang:iana-if-type">
    <nc:filter type="xpath" select="/if:interfaces/if:interface[if:type=derived-from-or-self(if:type, 'ianaift:ethernetCsmacd')]/enabled"/>
</nc:get>'''
response = m.dispatch(to_ele(message))
print_formatted(response.xml)

printing formatted str: <rpc-reply xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="urn:uuid:2cbaf50b-59f8-4e1c-9c69-e01906c22fd8"><data><interfaces xmlns="urn:ietf:params:xml:ns:yang:ietf-interfaces"><interface><name>LAN 1/0/4</name><enabled>true</enabled></interface><interface><name>LAN 1/0/5</name><enabled>true</enabled></interface><interface><name>LAN 1/0/7</name><enabled>true</enabled></interface><interface><name>LAN 1/0/8</name><enabled>true</enabled></interface><interface><name>LAN-DCN 1/0/0</name><enabled>true</enabled></interface><interface><name>WAN 1/1/1</name><enabled>true</enabled></interface></interfaces></data></rpc-reply>
<?xml version="1.0" ?>
<rpc-reply xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="urn:uuid:2cbaf50b-59f8-4e1c-9c69-e01906c22fd8">
	<data>
		<interfaces xmlns="urn:ietf:params:xml:ns:yang:ietf-interfaces">
			<interface>
				<name>LAN 1/0/4</name>
				<enabled>true</enabled>
			</interface>
			<interface>
				<name>LAN 1/0/5</name

In [8]:
message = '''<nc:get xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0">
  <nc:filter type="subtree">
    <if:interfaces xmlns:if="urn:ietf:params:xml:ns:yang:ietf-interfaces">
      <if:interface>
        <if:enabled/>
        <if:link-up-down-trap-enable/>
      </if:interface>
    </if:interfaces>
  </nc:filter>
</nc:get>'''
response = m.dispatch(to_ele(message))
print_formatted(response.xml)

printing formatted str: <rpc-reply xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="urn:uuid:c5de60fd-7bf4-4a50-a727-c0e9ce9083af"><data><interfaces xmlns="urn:ietf:params:xml:ns:yang:ietf-interfaces"><interface><name>CT 1/1/1</name><enabled>true</enabled><link-up-down-trap-enable>enabled</link-up-down-trap-enable></interface><interface><name>LAN 1/0/4</name><enabled>true</enabled><link-up-down-trap-enable>enabled</link-up-down-trap-enable></interface><interface><name>LAN 1/0/5</name><enabled>true</enabled><link-up-down-trap-enable>enabled</link-up-down-trap-enable></interface><interface><name>LAN 1/0/7</name><enabled>true</enabled><link-up-down-trap-enable>enabled</link-up-down-trap-enable></interface><interface><name>LAN 1/0/8</name><enabled>true</enabled><link-up-down-trap-enable>enabled</link-up-down-trap-enable></interface><interface><name>LAN-DCN 1/0/0</name><enabled>true</enabled><link-up-down-trap-enable>enabled</link-up-down-trap-enable></interface><interface><name>

Socket exception: An existing connection was forcibly closed by the remote host (10054)


# listing all bridges

In [16]:
message = '''
<nc:get xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0" xmlns:bp="urn:ieee:std:802.1Q:yang:ieee802-dot1q-bridge">
    <nc:filter type="xpath" select="bp:bridges"/>
</nc:get>'''
response = m.dispatch(to_ele(message))
print_formatted(response.xml)

printing formatted str: <rpc-reply xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="urn:uuid:29bd492a-cbba-4827-9e4b-7152aa6bcad3"><data><bridges xmlns="urn:ieee:std:802.1Q:yang:ieee802-dot1q-bridge"><bridge><name>BRIDGE</name><address>98-C5-DB-85-8D-5D</address><bridge-type>customer-vlan-bridge</bridge-type><components>1</components><component><name>DEFAULT</name><id>1</id><type>c-vlan-component</type><address>98-C5-DB-85-8D-5D</address><traffic-class-enabled>true</traffic-class-enabled><filtering-database><aging-time>300</aging-time><vlan-registration-entry><database-id>1</database-id><vids>1</vids><entry-type>static</entry-type><port-map><port-ref>1</port-ref><static-vlan-registration-entries><registrar-admin-control>fixed-new-ignored</registrar-admin-control><vlan-transmitted>untagged</vlan-transmitted></static-vlan-registration-entries></port-map><port-map><port-ref>3</port-ref><static-vlan-registration-entries><registrar-admin-control>fixed-new-ignored</registrar-admin

# listing bridges with name

In [6]:
message = '''
<nc:get xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0" xmlns:bp="urn:ieee:std:802.1Q:yang:ieee802-dot1q-bridge">
    <nc:filter type="xpath" select="bp:bridges/bp:bridge[name='BRIDGE']"/>
</nc:get>'''
response = m.dispatch(to_ele(message))
print_formatted(response.xml)

printing formatted str: <rpc-reply xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="urn:uuid:3c4aa618-d026-4fd2-a3e8-4286c381c62b"><data><bridges xmlns="urn:ieee:std:802.1Q:yang:ieee802-dot1q-bridge"><bridge><name>BRIDGE</name><address>98-A4-04-0C-33-F9</address><bridge-type>customer-vlan-bridge</bridge-type><components>1</components><component><name>DEFAULT</name><id>1</id><type>c-vlan-component</type><address>98-A4-04-0C-33-F9</address><traffic-class-enabled>true</traffic-class-enabled><filtering-database><aging-time>300</aging-time><vlan-registration-entry><database-id>1</database-id><vids>1</vids><entry-type>static</entry-type><port-map><port-ref>2</port-ref><static-vlan-registration-entries><registrar-admin-control>fixed-new-ignored</registrar-admin-control><vlan-transmitted>untagged</vlan-transmitted></static-vlan-registration-entries></port-map><port-map><port-ref>4</port-ref><static-vlan-registration-entries><registrar-admin-control>fixed-new-ignored</registrar-admin

In [19]:
message = '''<nc:get xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0" xmlns:bp="urn:ieee:std:802.1Q:yang:ieee802-dot1q-bridge">
    <nc:filter type="xpath" select="bp:bridges/bp:bridge[name='BRIDGE']/bp:component[name='POTATO']"/>
</nc:get> '''
response = m.dispatch(to_ele(message))
print_formatted(response.xml)

printing formatted str: <rpc-reply xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="urn:uuid:4c190cab-370a-4260-897f-efd1445c6c98"><data/></rpc-reply>
<?xml version="1.0" ?>
<rpc-reply xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="urn:uuid:4c190cab-370a-4260-897f-efd1445c6c98">
	<data/>
</rpc-reply>



# listing interfaces double filter

In [10]:
message = '''
<get-data xmlns="urn:ietf:params:xml:ns:yang:ietf-netconf-nmda">
  <datastore xmlns:ds="urn:ietf:params:xml:ns:yang:ietf-datastores">ds:operational</datastore>
  <xpath-filter xmlns:if="urn:ietf:params:xml:ns:yang:ietf-interfaces"
                xmlns:dot1q="urn:ieee:std:802.1Q:yang:ieee802-dot1q-bridge"
                xmlns:ianaift="urn:ietf:params:xml:ns:yang:iana-if-type">
      /if:interfaces/if:interface[if:type=derived-from-or-self(if:type, 'ianaift:ethernetCsmacd')][(name="LAN 1/2/4")]/oper-status
  </xpath-filter>
  <max-depth>unbounded</max-depth>
</get-data>
'''
response = m.dispatch(to_ele(message))
print_formatted(response.xml)

printing formatted str: <rpc-reply xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="urn:uuid:53884f9a-a66e-4adb-af77-9716089ffbfa"><data xmlns="urn:ietf:params:xml:ns:yang:ietf-netconf-nmda"><interfaces xmlns="urn:ietf:params:xml:ns:yang:ietf-interfaces"><interface><name>LAN 1/2/4</name><oper-status>up</oper-status></interface></interfaces></data></rpc-reply>
<?xml version="1.0" ?>
<rpc-reply xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="urn:uuid:53884f9a-a66e-4adb-af77-9716089ffbfa">
	<data xmlns="urn:ietf:params:xml:ns:yang:ietf-netconf-nmda">
		<interfaces xmlns="urn:ietf:params:xml:ns:yang:ietf-interfaces">
			<interface>
				<name>LAN 1/2/4</name>
				<oper-status>up</oper-status>
			</interface>
		</interfaces>
	</data>
</rpc-reply>



# listing interfaces all

In [7]:
message = '''
<get-data xmlns="urn:ietf:params:xml:ns:yang:ietf-netconf-nmda">
  <datastore xmlns:ds="urn:ietf:params:xml:ns:yang:ietf-datastores">ds:operational</datastore>
  <xpath-filter xmlns:if="urn:ietf:params:xml:ns:yang:ietf-interfaces"
                xmlns:dot1q="urn:ieee:std:802.1Q:yang:ieee802-dot1q-bridge"
                xmlns:ianaift="urn:ietf:params:xml:ns:yang:iana-if-type">
      /if:interfaces/if:interface[if:type=derived-from-or-self(if:type, 'ianaift:ethernetCsmacd')]
  </xpath-filter>
  <max-depth>unbounded</max-depth>
</get-data>
'''
response = m.dispatch(to_ele(message))
print_formatted(response.xml)

printing formatted str: <rpc-reply xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="urn:uuid:f49d8bd4-3b18-4c58-ac04-353e0f16c73f"><data xmlns="urn:ietf:params:xml:ns:yang:ietf-netconf-nmda"><interfaces xmlns="urn:ietf:params:xml:ns:yang:ietf-interfaces"><interface><name>LAN 1/2/4</name><description/><type xmlns:eiml="urn:rdns:com:ericsson:oammodel:ericsson-interface-mini-link">eiml:lanInterface</type><enabled>true</enabled><link-up-down-trap-enable>enabled</link-up-down-trap-enable><admin-status>up</admin-status><oper-status>up</oper-status><last-change>2024-10-14T20:13:31+02:00</last-change><if-index>2134638852</if-index><phys-address>98:c5:db:85:8d:67</phys-address><speed>1000000000</speed><statistics><in-octets>3199128353</in-octets><in-unicast-pkts>4180</in-unicast-pkts><in-broadcast-pkts>0</in-broadcast-pkts><in-multicast-pkts>45889138</in-multicast-pkts><in-discards>45607444</in-discards><in-errors>0</in-errors><out-octets>16954877</out-octets><out-unicast-pkts>0</out

In [4]:
message = '''
     <nc:rpc xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="45">
          <get/>
     </nc:rpc>
'''

response = m.dispatch(to_ele(message))
print_formatted(response.xml)

RPCError: Node "rpc" not found in the "ietf-netconf" module.

# vlan registration entry record

In [14]:
message = '''
<nc:edit-config xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0">
  <nc:target>
    <nc:running/>
  </nc:target>
  <nc:config>
    <dot1q:bridges xmlns:dot1q="urn:ieee:std:802.1Q:yang:ieee802-dot1q-bridge">
      <dot1q:bridge>
        <dot1q:name>BRIDGE</dot1q:name>
        <dot1q:component>
          <dot1q:name>DEFAULT</dot1q:name>
          <dot1q:bridge-vlan>
            <dot1q:vlan>
              <dot1q:vid>1234</dot1q:vid>
            </dot1q:vlan>
          </dot1q:bridge-vlan>
          <dot1q:filtering-database>
            <dot1q:vlan-registration-entry>
              <dot1q:database-id>1</dot1q:database-id>
              <dot1q:vids>1234</dot1q:vids>
              <dot1q:entry-type>static</dot1q:entry-type>
            </dot1q:vlan-registration-entry>
          </dot1q:filtering-database>
        </dot1q:component>
      </dot1q:bridge>
    </dot1q:bridges>
  </nc:config>
</nc:edit-config>
'''

response = m.dispatch(to_ele(message))
print_formatted(response.xml)

printing formatted str: <rpc-reply xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="urn:uuid:e565bbaa-e7a2-410c-a591-7a032058dc18"><ok/></rpc-reply>
<?xml version="1.0" ?>
<rpc-reply xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="urn:uuid:e565bbaa-e7a2-410c-a591-7a032058dc18">
	<ok/>
</rpc-reply>



# creating vlan without operation tag

In [70]:
message = '''
<nc:edit-config xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0">
  <nc:target>
    <nc:running/>
  </nc:target>
  <nc:config>
    <dot1q:bridges xmlns:dot1q="urn:ieee:std:802.1Q:yang:ieee802-dot1q-bridge">
      <dot1q:bridge>
        <dot1q:name>BRIDGE</dot1q:name>
        <dot1q:component>
          <dot1q:name>DEFAULT</dot1q:name>
          <dot1q:bridge-vlan>
            <dot1q:vlan>
              <dot1q:vid>1234</dot1q:vid>
            </dot1q:vlan>
          </dot1q:bridge-vlan>
          <dot1q:filtering-database>
            <dot1q:vlan-registration-entry>
              <dot1q:database-id>1</dot1q:database-id>
              <dot1q:vids>1234</dot1q:vids>
              <dot1q:entry-type>static</dot1q:entry-type>
              <dot1q:port-map>
                <dot1q:port-ref>2</dot1q:port-ref>
                <dot1q:static-vlan-registration-entries>
                  <dot1q:registrar-admin-control>fixed-new-ignored</dot1q:registrar-admin-control>
                  <dot1q:vlan-transmitted>tagged</dot1q:vlan-transmitted>
                </dot1q:static-vlan-registration-entries>
              </dot1q:port-map>
            </dot1q:vlan-registration-entry>
          </dot1q:filtering-database>
        </dot1q:component>
      </dot1q:bridge>
    </dot1q:bridges>
  </nc:config>
</nc:edit-config>
'''

response = m.dispatch(to_ele(message))
print_formatted(response.xml)

printing formatted str: <rpc-reply xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="urn:uuid:fc7f57eb-7215-4ebf-8c78-a8d16256b81e"><ok/></rpc-reply>
<?xml version="1.0" ?>
<rpc-reply xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="urn:uuid:fc7f57eb-7215-4ebf-8c78-a8d16256b81e">
	<ok/>
</rpc-reply>



# creating vlan and port mapping

In [66]:
message = '''
<nc:edit-config xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0">
  <nc:target>
    <nc:running/>
  </nc:target>
  <nc:config>
    <dot1q:bridges xmlns:dot1q="urn:ieee:std:802.1Q:yang:ieee802-dot1q-bridge">
      <dot1q:bridge>
        <dot1q:name>BRIDGE</dot1q:name>
        <dot1q:component>
          <dot1q:name>DEFAULT</dot1q:name>
          <dot1q:bridge-vlan>
            <dot1q:vlan nc:operation="create">
              <dot1q:vid>1234</dot1q:vid>
            </dot1q:vlan>
          </dot1q:bridge-vlan>
          <dot1q:filtering-database>
            <dot1q:vlan-registration-entry nc:operation="create">
              <dot1q:database-id>1</dot1q:database-id>
              <dot1q:vids>1234</dot1q:vids>
              <dot1q:entry-type>static</dot1q:entry-type>
              <dot1q:port-map nc:operation="create">
                <dot1q:port-ref>2</dot1q:port-ref>
                <dot1q:static-vlan-registration-entries>
                  <dot1q:registrar-admin-control>fixed-new-ignored</dot1q:registrar-admin-control>
                  <dot1q:vlan-transmitted>tagged</dot1q:vlan-transmitted>
                </dot1q:static-vlan-registration-entries>
              </dot1q:port-map>
            </dot1q:vlan-registration-entry>
          </dot1q:filtering-database>
        </dot1q:component>
      </dot1q:bridge>
    </dot1q:bridges>
  </nc:config>
</nc:edit-config>
'''

response = m.dispatch(to_ele(message))
print_formatted(response.xml)

RPCError: error: Node "port-map" to be created already exists.
error: Applying operation "create" failed.

# deleting bridge and registration entry and ports

In [62]:
message = '''
<nc:edit-config xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0">
  <nc:target>
    <nc:running/>
  </nc:target>
  <nc:config>
    <dot1q:bridges xmlns:dot1q="urn:ieee:std:802.1Q:yang:ieee802-dot1q-bridge">
      <dot1q:bridge>
        <dot1q:name>BRIDGE</dot1q:name>
        <dot1q:component>
          <dot1q:name>DEFAULT</dot1q:name>
          <dot1q:bridge-vlan>
            <dot1q:vlan nc:operation="delete">
              <dot1q:vid>1234</dot1q:vid>
            </dot1q:vlan>
          </dot1q:bridge-vlan>
          <dot1q:filtering-database>
            <dot1q:vlan-registration-entry nc:operation="delete">
              <dot1q:database-id>1</dot1q:database-id>
              <dot1q:vids>1234</dot1q:vids>
              <dot1q:port-map nc:operation="delete">
                <dot1q:port-ref>2</dot1q:port-ref>
              </dot1q:port-map>
            </dot1q:vlan-registration-entry>
          </dot1q:filtering-database>
        </dot1q:component>
      </dot1q:bridge>
    </dot1q:bridges>
  </nc:config>
</nc:edit-config>
'''

response = m.dispatch(to_ele(message))
print_formatted(response.xml)

RPCError: error: Node "port-map" to be deleted does not exist.
error: Applying operation "delete" failed.

# deleting bridge and registration entry 

In [63]:
message = '''
<nc:edit-config xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0">
  <nc:target>
    <nc:running/>
  </nc:target>
  <nc:config>
    <dot1q:bridges xmlns:dot1q="urn:ieee:std:802.1Q:yang:ieee802-dot1q-bridge">
      <dot1q:bridge>
        <dot1q:name>BRIDGE</dot1q:name>
        <dot1q:component>
          <dot1q:name>DEFAULT</dot1q:name>
          <dot1q:bridge-vlan>
            <dot1q:vlan nc:operation="delete">
              <dot1q:vid>1234</dot1q:vid>
            </dot1q:vlan>
          </dot1q:bridge-vlan>
          <dot1q:filtering-database>
            <dot1q:vlan-registration-entry nc:operation="delete">
              <dot1q:database-id>1</dot1q:database-id>
              <dot1q:vids>1234</dot1q:vids>
            </dot1q:vlan-registration-entry>
          </dot1q:filtering-database>
        </dot1q:component>
      </dot1q:bridge>
    </dot1q:bridges>
  </nc:config>
</nc:edit-config>
'''

response = m.dispatch(to_ele(message))
print_formatted(response.xml)

printing formatted str: <rpc-reply xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="urn:uuid:9c6fb0a3-96fe-472d-850c-db77c807f2d9"><ok/></rpc-reply>
<?xml version="1.0" ?>
<rpc-reply xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="urn:uuid:9c6fb0a3-96fe-472d-850c-db77c807f2d9">
	<ok/>
</rpc-reply>



# deleting only ports

In [71]:
message = '''
<nc:edit-config xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0">
  <nc:target>
    <nc:running/>
  </nc:target>
  <nc:config>
    <dot1q:bridges xmlns:dot1q="urn:ieee:std:802.1Q:yang:ieee802-dot1q-bridge">
      <dot1q:bridge>
        <dot1q:name>BRIDGE</dot1q:name>
        <dot1q:component>
          <dot1q:name>DEFAULT</dot1q:name>
          <dot1q:bridge-vlan>
            <dot1q:vlan >
              <dot1q:vid>1234</dot1q:vid>
            </dot1q:vlan>
          </dot1q:bridge-vlan>
          <dot1q:filtering-database>
            <dot1q:vlan-registration-entry >
              <dot1q:database-id>1</dot1q:database-id>
              <dot1q:vids>1234</dot1q:vids>
              <dot1q:port-map nc:operation="delete">
                <dot1q:port-ref>2</dot1q:port-ref>
              </dot1q:port-map>
            </dot1q:vlan-registration-entry>
          </dot1q:filtering-database>
        </dot1q:component>
      </dot1q:bridge>
    </dot1q:bridges>
  </nc:config>
</nc:edit-config>
'''

response = m.dispatch(to_ele(message))
print_formatted(response.xml)

printing formatted str: <rpc-reply xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="urn:uuid:7f9d6b4f-8081-497b-a756-8f43a1390bcf"><ok/></rpc-reply>
<?xml version="1.0" ?>
<rpc-reply xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="urn:uuid:7f9d6b4f-8081-497b-a756-8f43a1390bcf">
	<ok/>
</rpc-reply>



# deleting ports from vlan1 (default vlan)

In [12]:
message = '''
<nc:edit-config xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0">
  <nc:target>
    <nc:running/>
  </nc:target>
  <nc:config>
    <dot1q:bridges xmlns:dot1q="urn:ieee:std:802.1Q:yang:ieee802-dot1q-bridge">
      <dot1q:bridge>
        <dot1q:name>BRIDGE</dot1q:name>
        <dot1q:component>
          <dot1q:name>DEFAULT</dot1q:name>
          <dot1q:bridge-vlan>
            <dot1q:vlan >
              <dot1q:vid>1</dot1q:vid>
            </dot1q:vlan>
          </dot1q:bridge-vlan>
          <dot1q:filtering-database>
            <dot1q:vlan-registration-entry >
              <dot1q:database-id>1</dot1q:database-id>
              <dot1q:vids>1</dot1q:vids>
              <dot1q:port-map nc:operation="delete">
                <dot1q:port-ref>1</dot1q:port-ref>
              </dot1q:port-map>
            </dot1q:vlan-registration-entry>
          </dot1q:filtering-database>
        </dot1q:component>
      </dot1q:bridge>
    </dot1q:bridges>
  </nc:config>
</nc:edit-config>
'''

response = m.dispatch(to_ele(message))
print_formatted(response.xml)

printing formatted str: <rpc-reply xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="urn:uuid:d0ecc8a0-1d95-4efa-b1d9-9751d43213ef"><ok/></rpc-reply>
<?xml version="1.0" ?>
<rpc-reply xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="urn:uuid:d0ecc8a0-1d95-4efa-b1d9-9751d43213ef">
	<ok/>
</rpc-reply>

